# **3D Pose Estimation and Action Recognition**
### **Training Notebook**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys, json
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

from src.data import *
from src.models import *
from src.trainer import *
from src.visualizer import *

plt.style.use('ggplot')

In [3]:
# Bypass GPU memory limit
device = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(device[0], True)

IndexError: list index out of range

In [3]:
def conv(subject_paths):
    a = []
    for i in subject_paths:
        i = i.replace("\\","/")
        a.append(i)

    subject_paths = a
    return subject_paths

## **Data Pipeline**

In [13]:
# Load joints and action dictionary
joint_path = './src/mapper/all_joint_indices.json'
joint_indices = json.load(open(joint_path))
inv_joint_indices = {v: k for k, v in joint_indices.items()}

# Load action indices dictionary
action_indices_path = './src/mapper/action_indices.json'
action_indices = json.load(open(action_indices_path))
inv_action_indices = {v: k for k, v in action_indices.items()}

In [14]:
# Load data filepaths
train_path = 'preprocessed_train'
val_path = 'preprocessed_val'

# Get train dataset paths
pose_2d_paths_train = conv(sorted(glob(os.path.join('dataset', '3DPeople', train_path, '*', '*', 'pose_2d.txt'))))
pose_3d_paths_train = conv(sorted(glob(os.path.join('dataset', '3DPeople', train_path, '*', '*', 'pose_3d.txt'))))
action_paths_train = conv(sorted(glob(os.path.join('dataset', '3DPeople', train_path, '*', '*', 'action.txt'))))

# Get validation dataset paths
pose_2d_paths_val = conv(sorted(glob(os.path.join('dataset', '3DPeople', val_path, '*', '*', 'pose_2d.txt'))))
pose_3d_paths_val = conv(sorted(glob(os.path.join('dataset', '3DPeople', val_path, '*', '*', 'pose_3d.txt'))))
action_paths_val = conv(sorted(glob(os.path.join('dataset', '3DPeople', val_path, '*', '*', 'action.txt'))))

In [15]:
# Examine data count
print('Train dataset: {}'.format(len(pose_2d_paths_train)))
print('Val dataset: {}'.format(len(pose_2d_paths_val)))

Train dataset: 125420
Val dataset: 62710


In [16]:
# Create dataset object
batch_size = 256 

train_dataset = prepare_dataset(pose_2d_paths_train, pose_3d_paths_train, action_paths_train, batch_size)
val_dataset = prepare_dataset(pose_2d_paths_val, pose_3d_paths_val, action_paths_val, batch_size)

In [17]:
train_dataset

<PrefetchDataset shapes: ((256, 7, 2), (256, 7, 3), (256,)), types: (tf.float32, tf.float32, tf.int32)>

In [18]:
val_dataset

<PrefetchDataset shapes: ((256, 7, 2), (256, 7, 3), (256,)), types: (tf.float32, tf.float32, tf.int32)>

In [19]:
# Visualize dataset
for X, Y, Z in val_dataset:
    # Get action label
    print('Action Label: {}\n'.format(inv_action_indices[Z[100].numpy()]))
    # Visualize 2D pose
    visualize_2d(X[100].numpy())
    # Visualize 3D pose
    visualize_3d(Y[100].numpy())
    break

Action Label: 01_02_climb_down



## **Model Building**

In [20]:
# Build the regressor model
pose_regressor = PoseRegressor(num_joints = 7)

# Build the classifier model
action_classifier = ActionClassifier(input_shape = [7 * 3, 1], num_classes = 72)

# Build the multi-task model
MTN = MultiTaskPose(num_joints = 7, num_classes = 72)

In [21]:
print(pose_regressor)
print(action_classifier)
print(MTN)

In [22]:
params = {
    'regressor': pose_regressor,
    'classifier': action_classifier,
    'multi_task_net': MTN,

    'num_joints': 7,
    'lr_init': 0.001,
    'decay_rate': 0.96
}

In [23]:
print(params)

{'regressor': <tensorflow.python.keras.engine.functional.Functional object at 0x000001F39C6748B0>, 'classifier': <tensorflow.python.keras.engine.functional.Functional object at 0x000001F39FDD96D0>, 'multi_task_net': <tensorflow.python.keras.engine.functional.Functional object at 0x000001F39FDFD5B0>, 'num_joints': 7, 'lr_init': 0.001, 'decay_rate': 0.96}


In [24]:
# Examine model summary
pose_regressor.summary()
print('\n')
action_classifier.summary()
print('\n')
MTN.summary()

Model: "PoseRegressor"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 14)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         15360       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         1049600     dense[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1024)         4096        dense_1[0][0]                    
______________________________________________________________________________________

## **Experiment 1**
### Training two single-task network: Pose Regressor and Action Classifier

In [25]:
'''
Pre-train the pose-regressor
'''
# Initiate trainer
trainer = PoseTrainer(params)
num_epochs = 20

history_regressor_exp_1 = trainer.train_regressor(train_dataset, val_dataset, num_epochs = num_epochs)

[Epoch] 1/20 [Batch] 259/489 [Train Loss] 1.399705410003662 [Val Loss] Training....

KeyboardInterrupt: 

In [ ]:
'''
Train action classifier
'''
# Initiate trainer
trainer = PoseTrainer(params)

history_classifier_exp_1 = trainer.train_classifier(train_dataset, val_dataset, num_epochs = num_epochs)

In [ ]:
'''
Visualize output sample
'''
# Get sample batch
Xs, Ys, Zs = next(iter(val_dataset))

# Infer 3D pose
Ys_pred = pose_regressor(tf.reshape(Xs[100], [-1, params['num_joints'] * 2]), training = False)
Ys_pred = tf.reshape(Ys_pred, [-1, 3]).numpy()

# Infer action
Zs_pred = action_classifier(tf.reshape(Ys_pred, [-1, params['num_joints'] * 3]), training = False).numpy()
Zs_pred = np.argmax(Zs_pred, axis = 1)[0]

# Get action label
print('Action Label (GT)       : {}\n'.format(inv_action_indices[Zs[100].numpy()]))
print('Action Label (PREDICTED): {}\n'.format(inv_action_indices[Zs_pred]))

visualize_3d_compare(Ys[100].numpy(), Ys_pred)

In [ ]:
print(Xs)
print(Ys)
print(Zs)
print(Ys_pred)
print(Zs_pred)

In [ ]:
'''
Plot training loss & acc.
'''
# Regressor loss plot
plt.figure(dpi = 120)
plt.title('Regressor - Loss Plot')
plt.plot(history_regressor_exp_1['loss_train_regressor'], label = 'Train Loss')
plt.plot(history_regressor_exp_1['loss_val_regressor'], label = 'Validation Loss')
plt.legend(loc = 'center right')
plt.show()

# Classifier accuracy plot
plt.figure(dpi = 120)
plt.title('Classifier - Accuracy Plot')
plt.plot(history_classifier_exp_1['acc_train_classifier'], label = 'Train Acc.')
plt.plot(history_classifier_exp_1['acc_val_classifier'], label = 'Validation Acc.')
plt.legend(loc = 'center right')
plt.show()

In [ ]:
'''
Save results
'''
save_path = 'results/rendi/experiment_1'
os.makedirs(save_path, exist_ok = True)

# Save recorded metrics at training
train_metrics = np.zeros((num_epochs, 4))
train_metrics[:, 0] = history_regressor_exp_1['loss_train_regressor']
train_metrics[:, 1] = history_regressor_exp_1['loss_val_regressor']
train_metrics[:, 2] = history_classifier_exp_1['acc_train_classifier']
train_metrics[:, 3] = history_classifier_exp_1['acc_val_classifier']

np.savetxt(os.path.join(save_path, 'train_metrics.csv'), train_metrics,
           delimiter = ',',
           header = 'loss_train_regressor, loss_val_regressor, acc_train_classifier, acc_val_classifier')

# Save trained models
tf.keras.models.save_model(pose_regressor, filepath = os.path.join(save_path, 'pose_regressor.h5'), save_format = 'h5')
tf.keras.models.save_model(action_classifier, filepath = os.path.join(save_path, 'action_classifier.h5'), save_format = 'h5')

## **Experiment 2**
### Training one multi-task network

In [ ]:
# Initiate trainer
trainer = PoseTrainer(params)

In [ ]:
'''
Train Multi-task Network
'''
num_epochs = 20
history_MTN_exp_2 = trainer.train_MTN(train_dataset, val_dataset, num_epochs = num_epochs, train_option = 'both')

In [ ]:
'''
Visualize output sample
'''
# Get sample batch
Xs, Ys, Zs = next(iter(val_dataset))

# Infer 3D pose and action
Ys_pred, Zs_pred = MTN(tf.reshape(Xs[100], [-1, params['num_joints'] * 2]), training = False)

Ys_pred = tf.reshape(Ys_pred, [-1, 3]).numpy()
Zs_pred = np.argmax(Zs_pred, axis = 1)[0]

# Get action label
print('Action Label (GT)       : {}\n'.format(inv_action_indices[Zs[100].numpy()]))
print('Action Label (PREDICTED): {}\n'.format(inv_action_indices[Zs_pred]))

visualize_3d_compare(Ys[100].numpy(), Ys_pred)

In [ ]:
'''
Plot training loss & acc.
'''
# Regressor loss plot
plt.figure(dpi = 120)
plt.title('Regressor - Loss Plot')
plt.plot(history_MTN_exp_2['loss_train_regressor'], label = 'Train Loss')
plt.plot(history_MTN_exp_2['loss_val_regressor'], label = 'Validation Loss')
plt.legend(loc = 'center right')
plt.show()

# Classifier accuracy plot
plt.figure(dpi = 120)
plt.title('Classifier - Accuracy Plot')
plt.plot(history_MTN_exp_2['acc_train_classifier'], label = 'Train Acc.')
plt.plot(history_MTN_exp_2['acc_val_classifier'], label = 'Validation Acc.')
plt.legend(loc = 'center right')
plt.show()

In [ ]:
'''
Save results
'''
save_path = 'results/rendi/experiment_2'
os.makedirs(save_path, exist_ok = True)

# Save recorded metrics at training
train_metrics = np.zeros((num_epochs, 4))
train_metrics[:, 0] = history_MTN_exp_2['loss_train_regressor']
train_metrics[:, 1] = history_MTN_exp_2['loss_val_regressor']
train_metrics[:, 2] = history_MTN_exp_2['acc_train_classifier']
train_metrics[:, 3] = history_MTN_exp_2['acc_val_classifier']

np.savetxt(os.path.join(save_path, 'train_metrics.csv'), train_metrics,
           delimiter = ',',
           header = 'loss_train_regressor, loss_val_regressor, acc_train_classifier, acc_val_classifier')

# Save trained models
tf.keras.models.save_model(MTN, filepath = os.path.join(save_path, 'MTN_exp_2.h5'), save_format = 'h5')

## **Experiment 3**
### Hierarchical training of one multi-task network

In [ ]:
# Build the multi-task model
MTN = MultiTaskPose(num_joints = 7, num_classes = 72)

params = {
    'regressor': pose_regressor,
    'classifier': action_classifier,
    'multi_task_net': MTN,

    'num_joints': 7,
    'lr_init': 0.001,
    'decay_rate': 0.96
}

In [ ]:
'''
Train Multi-task Network (Pretraining regressor's head)
'''
# Initiate trainer
trainer = PoseTrainer(params)

num_epochs = 10
history_MTN_regressor_exp_3 = trainer.train_MTN(train_dataset, ,8 val_dataset, num_epochs = num_epochs, train_option = 'pose')

In [ ]:
'''
Train Multi-task Network (both tasks)
'''
# Initiate trainer
trainer = PoseTrainer(params)

num_epochs = 20
history_MTN_exp_3 = trainer.train_MTN(train_dataset, val_dataset, num_epochs = num_epochs, train_option = 'both')

In [ ]:
'''
Visualize output sample
'''
# Get sample batch
Xs, Ys, Zs = next(iter(val_dataset))

# Infer 3D pose and action
Ys_pred, Zs_pred = MTN(tf.reshape(Xs[100], [-1, params['num_joints'] * 2]), training = False)

Ys_pred = tf.reshape(Ys_pred, [-1, 3]).numpy()
Zs_pred = np.argmax(Zs_pred, axis = 1)[0]

# Get action label
print('Action Label (GT)       : {}\n'.format(inv_action_indices[Zs[100].numpy()]))
print('Action Label (PREDICTED): {}\n'.format(inv_action_indices[Zs_pred]))

visualize_3d_compare(Ys[100].numpy(), Ys_pred)

In [ ]:
'''
Plot training loss & acc.
'''
# Regressor loss plot
plt.figure(dpi = 120)
plt.title('Regressor - Loss Plot')
plt.plot(history_MTN_exp_3['loss_train_regressor'], label = 'Train Loss')
plt.plot(history_MTN_exp_3['loss_val_regressor'], label = 'Validation Loss')
plt.legend(loc = 'center right')
plt.show()

# Classifier accuracy plot
plt.figure(dpi = 120)
plt.title('Classifier - Accuracy Plot')
plt.plot(history_MTN_exp_3['acc_train_classifier'], label = 'Train Acc.')
plt.plot(history_MTN_exp_3['acc_val_classifier'], label = 'Validation Acc.')
plt.legend(loc = 'center right')
plt.show()

In [ ]:
'''
Save results
'''
save_path = 'results/rendi/experiment_3'
os.makedirs(save_path, exist_ok = True)

# Save recorded metrics at training
train_metrics = np.zeros((num_epochs, 4))
train_metrics[:, 0] = history_MTN_exp_3['loss_train_regressor']
train_metrics[:, 1] = history_MTN_exp_3['loss_val_regressor']
train_metrics[:, 2] = history_MTN_exp_3['acc_train_classifier']
train_metrics[:, 3] = history_MTN_exp_3['acc_val_classifier']

np.savetxt(os.path.join(save_path, 'train_metrics.csv'), train_metrics,
           delimiter = ',',
           header = 'loss_train_regressor, loss_val_regressor, acc_train_classifier, acc_val_classifier')

# Save trained models
tf.keras.models.save_model(MTN, filepath = os.path.join(save_path, 'MTN_exp_3.h5'), save_format = 'h5')

## **Quick Comparison between Experiments**

In [ ]:
# Regressor loss plot
plt.figure(dpi = 120)
plt.title('Regressor - Loss Plot')
plt.plot(history_regressor_exp_1['loss_val_regressor'], label = '[Exp. 1] Validation Loss')
plt.plot(history_MTN_exp_2['loss_val_regressor'], label = '[Exp. 2] Validation Loss')
plt.plot(history_MTN_exp_3['loss_val_regressor'], label = '[Exp. 3] Validation Loss')
plt.legend(loc = 'center right')
plt.show()

# Classifier accuracy plot
plt.figure(dpi = 120)
plt.title('Classifier - Accuracy Plot')
plt.plot(history_classifier_exp_1['acc_val_classifier'], label = '[Exp. 1] Validation Acc.')
plt.plot(history_MTN_exp_2['acc_val_classifier'], label = '[Exp. 2] Validation Acc.')
plt.plot(history_MTN_exp_3['acc_val_classifier'], label = '[Exp. 3] Validation Acc.')
plt.legend(loc = 'center right')
plt.show()